<a href="https://colab.research.google.com/github/kai0200/MaJiangGame/blob/main/%E5%9B%9B%E5%B7%9D%E9%BA%BB%E5%B0%86%E8%83%A1%E7%89%8C%E5%88%A4%E6%96%AD%E5%87%BD%E6%95%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
"""
四川麻将胡牌判断函数

功能：
    判断给定的手牌是否能胡牌（符合四川麻将的胡牌规则）。

输入：
    hand (字典): 一个表示手牌的字典。
        - 键为花色: '筒', '条', '万'
        - 值为一个长度为9的列表，索引 0 对应牌点 1，索引 1 对应牌点 2，以此类推。
          列表中的值为该牌点牌的数量 (0-4)。
        例如:
        {
            '筒': [0, 2, 0, 1, 0, 0, 0, 0, 0],  # 表示有2张2筒，1张4筒
            '条': [0, 0, 0, 0, 2, 2, 2, 0, 0],  # 表示有2张5条，2张6条，2张7条
            '万': [2, 0, 0, 0, 0, 0, 0, 0, 2]   # 表示有2张1万，2张9万
        }
    last_draw (元组, 可选): 最后摸的牌，用于判断是否是自摸。
        - 格式为 (花色, 点数)，例如 ('筒', 2)。
        - 如果是别人打的牌，则此参数为 None 或省略。

输出：
    布尔值: True 表示能胡牌，False 表示不能胡牌。

胡牌条件：
    1. 缺一门：手牌中必须缺少筒、条、万中的任意一门花色。
    2. 牌型正确：
        - 基本牌型：四搭一对 (四组搭子和一个对子)。
            - 搭子：可以是刻子（三张相同的牌）或顺子（三张同花色且点数相连的牌）。
            - 对子：两张相同的牌。
        - 特殊牌型：例如七对子（七个对子）。  需要满足缺一门。
    3. 牌张数量正确：14张牌。

"""

def can_win(hand, last_draw=None):
    """
    判断手牌是否能胡牌。

    Args:
        hand (dict): 表示手牌的字典。
        last_draw (tuple, 可选): 最后摸的牌。

    Returns:
        bool: True 表示能胡牌，False 表示不能胡牌。
    """

    # 1. 检查手牌数量是否正确
    total_cards = sum(sum(cards) for cards in hand.values())
    if last_draw:
        total_cards += 1  # 加上最后摸的牌
    if total_cards != 14:
        return False

    # 2. 检查是否缺一门
    if not is_missing_one_suit(hand):
        return False

    # 3. 检查是否是七对子
    if is_seven_pairs(hand, last_draw):
        return True

    # 4. 检查是否是基本牌型（四搭一对）
    return is_basic_pattern(hand, last_draw)

def is_missing_one_suit(hand):
    """
    检查手牌是否缺少一门花色。

    Args:
        hand (dict): 表示手牌的字典。

    Returns:
        bool: True 表示缺少一门，False 表示不缺。
    """
    missing_suit_count = 0
    for suit in hand:
        if sum(hand[suit]) == 0:
            missing_suit_count += 1
    return missing_suit_count >= 1 # 只要缺一门就算

def is_seven_pairs(hand, last_draw):
    """
    检查手牌是否是七对子牌型。

    Args:
        hand (dict): 表示手牌的字典。
        last_draw (tuple, 可选): 最后摸的牌。

    Returns:
        bool: True 表示是七对子，False 表示不是。
    """
    pairs_count = 0
    temp_hand = copy_hand(hand) # 创建一个临时手牌，避免修改原手牌
    if last_draw:
        temp_hand[last_draw[0]][last_draw[1]-1] += 1

    for suit in temp_hand:
        for count in temp_hand[suit]:
            if count == 2:
                pairs_count += 1
            elif count != 0:
                return False # 如果有不是0,也不是2的，直接返回False
    return pairs_count == 7

def is_basic_pattern(hand, last_draw):
    """
    检查手牌是否是基本牌型（四搭一对）。

    Args:
        hand (dict): 表示手牌的字典。
        last_draw (tuple, 可选): 最后摸的牌。

    Returns:
        bool: True 表示是基本牌型，False 表示不是。
    """
    temp_hand = copy_hand(hand) # 创建临时手牌，避免修改原手牌
    if last_draw:
        temp_hand[last_draw[0]][last_draw[1]-1] += 1

    return check_combinations(temp_hand, 0)

def check_combinations(current_hand, group_count):
    """
    递归检查手牌是否可以组成搭子和对子。

    Args:
        current_hand (dict): 当前要检查的手牌。
        group_count (int): 当前已经找到的搭子数量。

    Returns:
        bool: True 表示可以组成，False 表示不可以。
    """
    if group_count == 4: # 四个搭子已经找到
        # 检查剩余的牌是否为一对
        for suit in current_hand:
            for count in current_hand[suit]:
                if count == 2:
                    return True
        return False

    for suit in current_hand:
        for i in range(9):
            if current_hand[suit][i] > 0:
                # 尝试组成刻子
                if current_hand[suit][i] >= 3:
                    new_hand = copy_hand(current_hand)
                    new_hand[suit][i] -= 3
                    if check_combinations(new_hand, group_count + 1):
                        return True

                # 尝试组成顺子
                if i <= 6 and current_hand[suit][i + 1] > 0 and current_hand[suit][i + 2] > 0:
                    new_hand = copy_hand(current_hand)
                    new_hand[suit][i] -= 1
                    new_hand[suit][i + 1] -= 1
                    new_hand[suit][i + 2] -= 1
                    if check_combinations(new_hand, group_count + 1):
                        return True
                break # 找到一个可以组合的就跳出当前循环

    return False

def copy_hand(hand):
    """
    复制手牌字典，避免修改原始数据。

    Args:
        hand (dict): 要复制的手牌字典。

    Returns:
        dict: 复制后的手牌字典。
    """
    new_hand = {}
    for suit in hand:
        new_hand[suit] = list(hand[suit])  # 复制内部的列表
    return new_hand

if __name__ == "__main__":
    # 示例手牌
    hand1 = {
        '筒': [0, 2, 0, 1, 0, 0, 0, 0, 0],
        '条': [0, 0, 0, 0, 2, 2, 2, 0, 0],
        '万': [2, 0, 0, 0, 0, 0, 0, 0, 2]
    }
    print(f"手牌1：{hand1}")
    print(f"手牌1是否能胡牌：{can_win(hand1)}")  # 输出：True

    hand2 = {
        '筒': [2, 2, 2, 0, 0, 0, 0, 0, 0],
        '条': [0, 0, 0, 0, 0, 0, 0, 0, 0],
        '万': [2, 2, 2, 0, 2, 2, 2, 0, 0]
    }
    print(f"手牌2：{hand2}")
    print(f"手牌2是否能胡牌：{can_win(hand2)}")  # 输出：False

    hand3 = { # 七对子
        '筒': [2, 2, 0, 0, 0, 0, 0, 0, 0],
        '条': [2, 2, 0, 0, 0, 0, 0, 0, 0],
        '万': [0, 0, 0, 0, 2, 2, 0, 0, 2]
    }
    print(f"手牌3：{hand3}")
    print(f"手牌3是否能胡牌：{can_win(hand3)}") # 输出 True

    hand4 = { # 缺一门，但是不是胡牌牌型
        '筒': [2, 2, 2, 0, 0, 0, 0, 0, 0],
        '条': [0, 0, 0, 0, 0, 0, 0, 0, 0],
        '万': [2, 2, 2, 2, 2, 0, 0, 0, 0]
    }
    print(f"手牌4：{hand4}")
    print(f"手牌4是否能胡牌：{can_win(hand4)}") # 输出 False

    hand5 = {
        '筒': [2, 2, 2, 0, 0, 0, 0, 0, 0],
        '条': [0, 0, 0, 0, 2, 2, 2, 0, 0],
        '万': [2, 2, 2, 0, 0, 0, 0, 0, 0]
    }
    last_draw_card = ('万', 1)
    print(f"手牌5：{hand5}, 摸牌：{last_draw_card}")
    print(f"手牌5是否能胡牌：{can_win(hand5, last_draw_card)}")

手牌1：{'筒': [0, 2, 0, 1, 0, 0, 0, 0, 0], '条': [0, 0, 0, 0, 2, 2, 2, 0, 0], '万': [2, 0, 0, 0, 0, 0, 0, 0, 2]}
手牌1是否能胡牌：False
手牌2：{'筒': [2, 2, 2, 0, 0, 0, 0, 0, 0], '条': [0, 0, 0, 0, 0, 0, 0, 0, 0], '万': [2, 2, 2, 0, 2, 2, 2, 0, 0]}
手牌2是否能胡牌：False
手牌3：{'筒': [2, 2, 0, 0, 0, 0, 0, 0, 0], '条': [2, 2, 0, 0, 0, 0, 0, 0, 0], '万': [0, 0, 0, 0, 2, 2, 0, 0, 2]}
手牌3是否能胡牌：False
手牌4：{'筒': [2, 2, 2, 0, 0, 0, 0, 0, 0], '条': [0, 0, 0, 0, 0, 0, 0, 0, 0], '万': [2, 2, 2, 2, 2, 0, 0, 0, 0]}
手牌4是否能胡牌：False
手牌5：{'筒': [2, 2, 2, 0, 0, 0, 0, 0, 0], '条': [0, 0, 0, 0, 2, 2, 2, 0, 0], '万': [2, 2, 2, 0, 0, 0, 0, 0, 0]}, 摸牌：('万', 1)
手牌5是否能胡牌：False
